In [1]:
from dotenv import load_dotenv

load_dotenv('../../.env')


True

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

# Load Documents

In [3]:
import requests
import tqdm
from typing import List

# financial reports of amamzon, but can be replaced by any URLs of pdfs
urls = ['https://s2.q4cdn.com/299287126/files/doc_financials/Q1_2018_-_8-K_Press_Release_FILED.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/Q2_2018_Earnings_Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Q318-Amazon-Earnings-Press-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/AMAZON.COM-ANNOUNCES-FOURTH-QUARTER-SALES-UP-20-TO-$72.4-BILLION.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/Q119_Amazon_Earnings_Press_Release_FINAL.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q2-2019-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Q3-2019-Amazon-Financial-Results.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q4-2019-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q1/Amazon-Q1-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q2/AMZN-Q2-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q3/Q3-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q4/business_and_financial_update.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q1/Q1-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q2/Q2-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q3/Q3-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/Q4-2022-Amazon-Earnings-Release.pdf'
        ]

def load_reports(urls: List[str]) -> List[str]:
    """ Load pages from a list of urls"""
    pages = []

    for url in tqdm.tqdm(urls):
        r = requests.get(url)
        path = url.split('/')[-1]
        with open(path, 'wb') as f:
            f.write(r.content)
        loader = PyPDFLoader(path)
        local_pages = loader.load_and_split()
        pages.extend(local_pages)
    return pages

pages = load_reports(urls)



  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [02:19<00:00,  7.33s/it]


# Use Text Splitters

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs = {'device':'cpu'} )

db = DeepLake(dataset_path="hub://thapabibek1129/amazon_earnings_6", embedding_function=embeddings)
db.add_documents(texts)


Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://thapabibek1129/amazon_earnings_6 already exists, loading from the storage


Creating 414 embeddings in 1 batches of size 414:: 100%|██████████| 1/1 [02:03<00:00, 123.99s/it]

Dataset(path='hub://thapabibek1129/amazon_earnings_6', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (828, 384)  float32   None   
    id        text      (828, 1)     str     None   
 metadata     json      (828, 1)     str     None   
   text       text      (828, 1)     str     None   


['00302480-caf8-11ee-9f56-a434d9523559',
 '00302481-caf8-11ee-b9dc-a434d9523559',
 '00302482-caf8-11ee-a92c-a434d9523559',
 '00302483-caf8-11ee-a6dc-a434d9523559',
 '00302484-caf8-11ee-945f-a434d9523559',
 '00302485-caf8-11ee-9f0d-a434d9523559',
 '00302486-caf8-11ee-94d9-a434d9523559',
 '00302487-caf8-11ee-8c50-a434d9523559',
 '00302488-caf8-11ee-bbc5-a434d9523559',
 '00302489-caf8-11ee-bd07-a434d9523559',
 '0030248a-caf8-11ee-9057-a434d9523559',
 '0030248b-caf8-11ee-8ab0-a434d9523559',
 '0030248c-caf8-11ee-9d9d-a434d9523559',
 '0030248d-caf8-11ee-9c14-a434d9523559',
 '0030248e-caf8-11ee-beef-a434d9523559',
 '0030248f-caf8-11ee-aa5d-a434d9523559',
 '00302490-caf8-11ee-95fb-a434d9523559',
 '00302491-caf8-11ee-9529-a434d9523559',
 '00302492-caf8-11ee-951c-a434d9523559',
 '00302493-caf8-11ee-8bcc-a434d9523559',
 '00302494-caf8-11ee-8890-a434d9523559',
 '00302495-caf8-11ee-a886-a434d9523559',
 '00302496-caf8-11ee-8836-a434d9523559',
 '00302497-caf8-11ee-938a-a434d9523559',
 '00302498-caf8-

# Create a Retrieval Chain

In [20]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    model_kwargs={'temperature':0.5,"max_length": 64,"max_new_tokens":512}
)



In [21]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=db.as_retriever())

In [22]:
qa("Combine total revenue in 2020?")

{'query': 'Combine total revenue in 2020?',
 'result': ' The total revenue for Amazon.com Inc. in 2020 was $221,598 million. This can be calculated by adding the net sales for the North America, International, and AWS segments for the full year 2020: $55,436 million + $83,935 million + $84,227 million = $223,604 million. However, the provided financial statements show a total net sales figure of $214,606 million for the year 2020. The discrepancy might be due to rounding or the exclusion of certain items in the given figures. Therefore, the closest approximation from the provided data is $214,606 million.'}

In [23]:
print(qa("What is the revenue in 2021 Q3?"))

{'query': 'What is the revenue in 2021 Q3?', 'result': ' The revenue in 2021 Q3 was $82,360 million.\nExplanation: The revenue for 2021 Q3 can be found in the "North America Segment" table under the "Net sales" column for Q3 2021. The number is $82,360 million.'}


In [24]:
qa("What is the revenue in 2023 Q1?")

{'query': 'What is the revenue in 2023 Q1?',
 'result': ' Based on the information provided in the text, the revenue for Amazon.com in 2023 Q1 is expected to be between $121.0 billion and $126.0 billion.\n\nExplanation: The text states that net sales for Amazon.com in 2023 Q1 are expected to be between $121.0 billion and $126.0 billion. This range represents the expected revenue for the company during this quarter.'}